# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.
There are two options for communcating with Julia: `PyCall.jl/pyjulia` and `PythonCall.jl/juliacall`.
Here we use the first by passing `calljulia="pyjulia"` when initializing.

In [1]:
import qiskit_alt
qiskit_alt.project.ensure_init(calljulia="pyjulia", compile=False, depot=True)

We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
julia = qiskit_alt.project.julia
julia.Main.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into much of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['YYYY', 'XXYY', 'YYXX', 'XXXX', 'IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ'],
              coeffs=[ 0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
 -0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
import qiskit_alt.electronic_structure
fermi_op = qiskit_alt.electronic_structure.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.electronic_structure.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'XXXX', 'YYXX', 'XXYY', 'YYYY', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j])

Note that the constant term differs. The qiskit-nature version ignores the nuclear-repulsion term. I need to open an issue about whether and how to handle it.

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 3.37 ms
geometry=h2_geometry, basis='631g' 39.25 ms
geometry=h2_geometry, basis='631++g' 209.80 ms
geometry=h2o_geometry, basis='sto3g' 246.12 ms
geometry=h2o_geometry, basis='631g' 3004.95 ms


[3.371502598747611,
 39.25394311081618,
 209.79926399886608,
 246.11722519621253,
 3004.945662105456]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 1.30 ms
geometry=h2_geometry, basis='631g' 3.44 ms
geometry=h2_geometry, basis='631++g' 24.52 ms
geometry=h2o_geometry, basis='sto3g' 18.70 ms
geometry=h2o_geometry, basis='631g' 469.48 ms


[1.3044350082054734,
 3.4391670022159815,
 24.519264418631792,
 18.69659200310707,
 469.4802389945835]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[2.5846459022790462,
 11.413793830169755,
 8.556507259632268,
 13.163747979060131,
 6.400579646420698]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 65.00 ms
geometry=h2_geometry, basis='631g' 105.01 ms
geometry=h2_geometry, basis='631++g' 298.17 ms
geometry=h2o_geometry, basis='sto3g' 438.85 ms
geometry=h2o_geometry, basis='631g' 24030.27 ms


[65.00064351130277,
 105.00940189231187,
 298.17473497241735,
 438.84517257101834,
 24030.2699431777]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 59.39 ms
geometry=h2_geometry, basis='631g' 62.03 ms
geometry=h2_geometry, basis='631++g' 67.03 ms
geometry=h2o_geometry, basis='sto3g' 77.55 ms
geometry=h2o_geometry, basis='631g' 193.82 ms


[59.38674798235297,
 62.029914115555584,
 67.0257663121447,
 77.55265950690955,
 193.81799241527915]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.094531115437033,
 1.6928832385079522,
 4.448658350040971,
 5.658673414441957,
 123.9836902844905]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.com/Qiskit-Extensions/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.com/Qiskit-Extensions/ElectronicStructure.jl) provides an interface to electronic structure packages.

* [`ElectronicStructurePySCF.jl`](https://github.com/Qiskit-Extensions/ElectronicStructurePySCF.jl) provides an interface to `pyscf`

* [`QiskitQuantumInfo.jl`](https://github.com/Qiskit-Extensions/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* [Issue](https://github.com/Qiskit-Extensions/QuantumOps.jl/issues/17) for improving performance of Jordan-Wigner in `QuantumOps`.
    * Precompute one and two-body terms
    * Use @threads


# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt.pauli_operators import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.47494296+0.00000000e+00j,  0.40823308+0.00000000e+00j,
  0.        +8.72050729e-03j,  0.08853865+0.00000000e+00j,
  0.48022917+0.00000000e+00j,  0.50427456+0.00000000e+00j,
  0.        +1.86142504e-01j,  0.06825203+0.00000000e+00j,
  0.        -1.20594387e-01j,  0.        +1.63950963e-01j,
  0.0933073 +0.00000000e+00j,  0.        +3.95169450e-02j,
 -0.08316598+0.00000000e+00j, -0.00272138+0.00000000e+00j,
  0.        -7.34809038e-02j,  0.06674014+0.00000000e+00j,
  0.58852484+0.00000000e+00j,  0.51606481+0

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.23 ms
nqubits=3, 5.00 ms
nqubits=4, 24.66 ms
nqubits=5, 142.61 ms
nqubits=6, 1053.21 ms
nqubits=7, 8999.82 ms
nqubits=8, 107648.63 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 1.29 ms
nqubits=3, 1.36 ms
nqubits=4, 11.33 ms
nqubits=5, 27.75 ms
nqubits=6, 27.85 ms
nqubits=7, 107.10 ms
nqubits=8, 567.22 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[0.9556047308329915,
 3.67816523510366,
 2.1765062775946946,
 5.138368881804313,
 37.8119442420315,
 84.0323151825261,
 189.7838578070803]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.5427255993708968 ms
k=10, n=100, 1.8118745530955493 ms
k=10, n=1000, 16.81242184713483 ms
k=10, n=5000, 83.37073109578341 ms
k=10, n=10000, 167.40924795158207 ms
k=10, n=100000, 1695.4269821289927 ms
k=100, n=10, 0.336431001778692 ms
k=100, n=100, 1.897125702816993 ms
k=100, n=1000, 17.57452649762854 ms
k=100, n=5000, 87.75187629507855 ms
k=100, n=10000, 176.62636120803654 ms
k=100, n=100000, 1799.1630749311298 ms


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 1.2364529073238373 ms
k=10, n=100, 1.3390239444561303 ms
k=10, n=1000, 2.580544597003609 ms
k=10, n=5000, 8.644396206364036 ms
k=10, n=10000, 25.033850746694952 ms
k=10, n=100000, 228.47767625935376 ms
k=100, n=10, 5.032895249314606 ms
k=100, n=100, 1.4746043947525322 ms
k=100, n=1000, 3.6469499580562115 ms
k=100, n=5000, 17.004536895547062 ms
k=100, n=10000, 33.38807285763323 ms
k=100, n=100000, 370.6562086008489 ms


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.43893754154015063,
 1.3531308088978768,
 6.515067349216332,
 9.644482865605532,
 6.687315093691049,
 7.420536701381927,
 0.06684641446183648,
 1.286531973977582,
 4.818965628745723,
 5.16049786207691,
 5.2901035037623005,
 4.853994168133864]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia takes all the time.

We see this in the following.

In [20]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


Generate `1000` ten-qubit Pauli strings.

In [21]:
%julia using Random: randstring
%julia pauli_strings = [randstring("IXYZ", 10) for _ in 1:1000]
None;

Benchmark converting these to a `QuantumOps.PauliSum`. Note that as the sums are always sorted.

In [22]:
%julia import Pkg; Pkg.add("BenchmarkTools")
%julia using BenchmarkTools: @btime
%julia @btime QuantumOps.PauliSum($pauli_strings)
None;

    Updating registry at `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.7.2/depot/registries/QuantumRegistry`
    Updating git-repo `https://github.com/Qiskit-Extensions/QuantumRegistry`
    Updating registry at `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.7.2/depot/registries/General.toml`
   Resolving package versions...
  No Changes to `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.7.2/Project.toml`
  No Changes to `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.7.2/Manifest.toml`


  341.921 μs (1012 allocations: 134.17 KiB)


Check that we are actually getting the right result.

In [23]:
%julia pauli_sum = QuantumOps.PauliSum(pauli_strings);
%julia println(length(pauli_sum))
%julia println(pauli_sum[1])

999
10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
IIIIXZZIZZ * (1 + 0im)


In [24]:
6.9 * 2.29 / .343  # Ratio of time to construct PauliSum via qiskit_alt to time in pure Julia

46.06705539358601

So the pure Julia code is `46` times faster than the qiskit.quantum_info.
**But, the `qiskit.quantum_info` is also extracting a possible phase !**

In [25]:
import qiskit.tools.jupyter
d = qiskit.__qiskit_version__._version_dict
d['qiskit_alt'] = qiskit_alt.__version__
%qiskit_version_table

/home/lapeyre/julia_qiskit_repos/qiskit_alt/venv/lib/python3.9/site-packages/qiskit/tools/jupyter/__init__.py:133: RuntimeWarning: matplotlib can't be found, ensure you have matplotlib and other visualization dependencies installed. You can run '!pip install qiskit-terra[visualization]' to install it from jupyter
  warnings.warn(
